**Introduction to the `sqlite3` Library**
========================================================


**Relevant Links:**
---------------

*  Python `sqlite3` library documentation: https://docs.python.org/3/library/sqlite3.html#module-sqlite3   
*  `SQLite` Documentation: https://www.sqlite.org/docs.html   
*  Well-known users of `SQLite`: https://www.sqlite.org/famous.html  

**`SQLite`** is a C library that provides a lightweight disk-based database that doesn’t require a separate server process. Applications can use SQLite for internal data storage. It’s also possible to prototype an application using SQLite and then later port the application to a production database system.   

`SQLite` databases are stored in a file on disk (usually with a `.db` extension). If you attempt to connect to a database file that doesn't exist, `SQLite` with create a new database, assign it the name you passed to the connect function and save it to your current working directory.

Typical sqlite setup and usage in Python is as follows:

*  **Create a connection object** - For `SQLite`, pass the database filepath instead of authentication details:

            db = sqlite3.connect(<filename>.db)
            
*  **Define a cursor** - Call the `cursor()` method on the connection object created above:

            cursor = db.cursor()  
            
*  **Construct a query for the dataset of interest** - This can be almost any valid SQL command:

            SQL = "SELECT * FROM SAMPLE_TABLE"  
            
*  **Pass the query string to the cursor's `execute` method:**

            cursor.execute(SQL)
            
            
*  **Iterate over the cursor** - Using a `for` loop, list comprehensions, etc...  

  
The result will be a list of tuples, so data elements can be accessed by row or selectively by referencing components by index offset:

    for record in cursor: print(record)

**Creating Databases and Tables**
--------------------

If the database file passed to the `sqlite3.connect` method doesn't exist, a new database with the name specified will be created. The following example creates a database consisting of 2 tables: The first table holds closing stock prices, the second contains a mapping between ticker symbols and company names:

NOTE: For more information on `SQLite` datatypes and the resulting affinity mappings of common datatypes for   
other RDBMS, follow this link and navigate to the *Affinity Name Examples* section:  

https://www.sqlite.org/datatype3.html

In [1]:
!pip install SQLAlchemy==1.4.4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 47.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for SQLAlchemy: filename=SQLAlchemy-1.4.4-cp311-cp311-linux_x86_64.whl size=1499388 sha256=c4821165a0ef89246ed1039f0fe5fd5fe5390c93d71e1fbb7f0d35ee29480147
  Stored in directory: /root/.cache/pip/wheels/23/af/5b/7c877cbf6f4b966e013fdb3ef2044a57bec7a08da7d7d9fb7a
Successfully built SQLAlchemy
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.41
    Uninstalling SQLAlchemy-2.0.41:
      Successfully uninstalled SQLAlchemy-2.0.41
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython-sql 0.5.0 requires sqlalchemy>=2.0, but you have sqlalchemy 1.4.4 which is incompatible.


In [2]:
"""
Creating a new database with two tables using the `sqlite3`
module.

=============================
Table 1                     |
=============================
TABLENAME: `CLOSING_PRICES` |
                            |
FIELDS   : DATE   TEXT      |
           TICKER TEXT      |
           CLOSE  REAL      |
=============================

=============================
Table 2                     |
=============================
TABLENAME: `TICKER_MAPPING` |
                            |
FIELDS   : TICKER       TEXT|
           COMPANY NAME TEXT|
=============================

"""
import sqlite3

# Create new database `sample.db`. Notice `sample.db` is now
# listed in your working directory:
db = sqlite3.connect("sample.db")

# Initiate a cursor, and call the connection's cursor method:
cursor = db.cursor()

# Specify the DDL to create the two tables:
tbl1_ddl = """CREATE TABLE CLOSING_PRICES (
              DATE   TEXT,
              TICKER TEXT,
              CLOSE  REAL)"""

tbl2_ddl = """CREATE TABLE TICKER_MAPPING (
              TICKER       TEXT,
              COMPANY_NAME TEXT)"""

# Call the `cursor.execute` method, passing tbl1_ddl & tbl2_ddl as arguments:
cursor.execute(tbl1_ddl)
cursor.execute(tbl2_ddl)

# IMPORTANT! Be sure to commit changes you want to persist. Without
# commiting, changes will not be saved:
db.commit()

# close connection to `sample.db`:
db.close()


In [3]:
db = sqlite3.connect("sample.db")

# Initiate a cursor, and call the connection's cursor method:
cursor = db.cursor()

# Specify the DDL to create the two tables:
cursor.execute( """SELECT * FROM  CLOSING_PRICES """)
print(cursor.fetchall())
db.close()


[]


To verify that your tables have been created, run the following:

In [4]:
# Restablish connection to `sample.db`:
db     = sqlite3.connect('sample.db')
cursor = db.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
print(cursor.fetchall())
db.close()

[('CLOSING_PRICES',), ('TICKER_MAPPING',)]


**Loading Data into `SQLite` Database Tables**
--

The following example demonstrates two methods of loading data into `SQLite` tables. The first method assumes the data is already available in memory in the current Python session. The second method assumes data is being loaded from a .csv file:

NOTE: The file `ticker_data.csv`, included with this notebook, is required for the 2nd example.

In [5]:
#==========================================================
# Method #1: Data already avialable in Python session     |
#==========================================================

# Reestablish connection to `sample.db` database:
db     = sqlite3.connect('sample.db')
cursor = db.cursor()

# Insert four records into `CLOSING_PRICES` table based on the closing
# prices of AXP, GE, GS & UTX on 7.22.2016.

# Single records can be inserted using the `cursor.execute` method:
cursor.execute("INSERT INTO TICKER_MAPPING VALUES ('AXP',  'American Express Company')")
cursor.execute("INSERT INTO TICKER_MAPPING VALUES ('GS', 'GenSeral Electric Company')")
cursor.execute("INSERT INTO TICKER_MAPPING VALUES ('GE' ,  'Goldman Sachs Group Inc')")
cursor.execute("INSERT INTO TICKER_MAPPING VALUES ('UTX' , 'United Technologies Corporation')")


# We can insert several records at once if we create a list
# of tuples of the data to insert, then call `cursor.executemany`:
closing_prices = [('20160722', 'AXP',  64.28),
                  ('20160722', 'GE' ,  32.06),
                  ('20160722', 'GS' , 160.41),
                  ('20160722', 'UTX', 105.13)]


# Calling `cursor.executemany`
cursor.executemany("INSERT INTO CLOSING_PRICES VALUES (?,?,?)", closing_prices)

# Not forgetting to commit changes and close connection:
db.commit()
db.close()

In [6]:
db     = sqlite3.connect('sample.db')
cursor = db.cursor()
cursor.execute("SELECT * FROM CLOSING_PRICES")
print(cursor.fetchall())
db.close()

[('20160722', 'AXP', 64.28), ('20160722', 'GE', 32.06), ('20160722', 'GS', 160.41), ('20160722', 'UTX', 105.13)]


The `(?,?,?)` in `cursor.executemany` serve as placeholders for columns in the target table. There should be one `?` for each column in the target table. The more common scenario may be loading data from file into an `SQLite` database table. The syntax is similiar, with added file handling logic:

In [8]:
#=====================================
# Method #2: Data read in from .csv  |
#=====================================

# Requires `ticker_data.csv` file
import sqlite3
import csv

# Reestablish connection to `sample.db` database:
db     = sqlite3.connect('sample.db')
cursor = db.cursor()

# Open `ticker_data.csv`, and create a csv.reader instance:
with open('ticker_data.csv', 'r', encoding='cp1252') as f:  # on Windows, change `r` to `rb`

    fcsv = csv.reader(f)

    # `recs_to_load` is a list of records contained in `ticker_data.csv`:
    recs_to_load = [record for record in fcsv]

    # call `cursor.executemany`, specifying `recs_to_load`
    cursor.executemany("INSERT INTO CLOSING_PRICES VALUES (?,?,?)", recs_to_load)

# Not forgetting to commit changes and close connection:
db.commit()
db.close()

In [9]:
db     = sqlite3.connect('sample.db')
cursor = db.cursor()
cursor.execute("SELECT * FROM CLOSING_PRICES")
print(cursor.fetchall())
db.close()

[('20160722', 'AXP', 64.28), ('20160722', 'GE', 32.06), ('20160722', 'GS', 160.41), ('20160722', 'UTX', 105.13), ('DATE', 'TICKER', 'CLOSE'), ('20161125', 'GE', 31.44), ('20161123', 'GE', 31.34), ('20161122', 'GE', 31.18), ('20161121', 'GE', 30.87), ('20161118', 'GE', 30.67), ('20161117', 'GE', 30.79), ('20161116', 'GE', 30.74), ('20161115', 'GE', 30.75), ('20161114', 'GE', 30.51), ('20161111', 'GE', 30.71), ('20161110', 'GE', 30.41), ('20161109', 'GE', 29.63), ('20161108', 'GE', 29.42), ('20161107', 'GE', 29.31), ('20161104', 'GE', 28.44), ('20161103', 'GE', 28.28), ('20161102', 'GE', 28.49), ('20161101', 'GE', 28.88), ('20161031', 'GE', 29.1), ('20161028', 'GE', 29.22), ('20161027', 'GE', 28.63), ('20161026', 'GE', 28.87), ('20161025', 'GE', 28.65)]


**`SQLite` Data Retrieval**
==================

Retrieval of `SQLite` database records is carried out in exactly the same as with `cx_Oracle`. An iterator in the form  
of a database cursor is returned, which is traversed, yielding the query is question's result set:

In [10]:
import sqlite3

# reestablish connection to `sample.db` database:
db     = sqlite3.connect('sample.db')
cursor = db.cursor()

#construct a query to retrieve data from `CLOSING_PRICES`:
SQL = "SELECT * FROM CLOSING_PRICES"

# call `cursor.execute` on query string:
cursor.execute(SQL)

# `cursor` can be iterated over:
for rec in cursor:
    print(rec)

# not forgetting to commit changes and close connection:
db.commit()
db.close()


('20160722', 'AXP', 64.28)
('20160722', 'GE', 32.06)
('20160722', 'GS', 160.41)
('20160722', 'UTX', 105.13)
('DATE', 'TICKER', 'CLOSE')
('20161125', 'GE', 31.44)
('20161123', 'GE', 31.34)
('20161122', 'GE', 31.18)
('20161121', 'GE', 30.87)
('20161118', 'GE', 30.67)
('20161117', 'GE', 30.79)
('20161116', 'GE', 30.74)
('20161115', 'GE', 30.75)
('20161114', 'GE', 30.51)
('20161111', 'GE', 30.71)
('20161110', 'GE', 30.41)
('20161109', 'GE', 29.63)
('20161108', 'GE', 29.42)
('20161107', 'GE', 29.31)
('20161104', 'GE', 28.44)
('20161103', 'GE', 28.28)
('20161102', 'GE', 28.49)
('20161101', 'GE', 28.88)
('20161031', 'GE', 29.1)
('20161028', 'GE', 29.22)
('20161027', 'GE', 28.63)
('20161026', 'GE', 28.87)
('20161025', 'GE', 28.65)


Headers need to be extracted from the `cursor.description` attribute:

In [11]:
# Obtain reference to table headers:
import sqlite3

# Reestablish connection to `sample.db` database:
db     = sqlite3.connect('sample.db')
cursor = db.cursor()

#construct a query to retrieve data from `CLOSING_PRICES`:
SQL = "SELECT * FROM CLOSING_PRICES"

# Call `cursor.execute` on SQL:
cursor.execute(SQL)

# Capture table headers into `headers` list:
headers = [i[0] for i in cursor.description]

# Not forgetting to commit changes and close connection:
db.commit()
db.close()

print("CLOSING_PRICES table fieldnames: {}".format(headers))



CLOSING_PRICES table fieldnames: ['DATE', 'TICKER', 'CLOSE']


The following is an example using bind variables with `sqlite`:

In [12]:
import sqlite3

# Reestablish connection to `sample.db` database:
db     = sqlite3.connect('sample.db')
cursor = db.cursor()

# Bind variable key-value pairs:
params = {'symbol':'GE','date':'20161125'}
SQL    = "SELECT * FROM CLOSING_PRICES WHERE TICKER=:symbol AND DATE!=:date"
cursor.execute(SQL, params)

headers = [i[0] for i in cursor.description]

#returns records where ticker == 'GE' && date != '20161125'
print(headers)
for record in cursor: print(*record)

db.commit()
db.close()


['DATE', 'TICKER', 'CLOSE']
20160722 GE 32.06
20161123 GE 31.34
20161122 GE 31.18
20161121 GE 30.87
20161118 GE 30.67
20161117 GE 30.79
20161116 GE 30.74
20161115 GE 30.75
20161114 GE 30.51
20161111 GE 30.71
20161110 GE 30.41
20161109 GE 29.63
20161108 GE 29.42
20161107 GE 29.31
20161104 GE 28.44
20161103 GE 28.28
20161102 GE 28.49
20161101 GE 28.88
20161031 GE 29.1
20161028 GE 29.22
20161027 GE 28.63
20161026 GE 28.87
20161025 GE 28.65
